In [2]:
import os
import re
import csv
import uuid

In [4]:
# Variables
source_directory = "source_files/"
output_directory = "extracted_tables/"
table_code_directory = output_directory + "table_code/"
table_header_directory = output_directory + "table_header/"
result_file = "tables_regex.csv"

if os.path.isdir(output_directory) == False:
    os.mkdir(output_directory)

if os.path.isdir(table_code_directory) == False:
    os.mkdir(table_code_directory)
    
if os.path.isdir(table_header_directory) == False:
    os.mkdir(table_header_directory)

In [22]:
# TODO: Save paper name and authors as metadata
output_file_path = output_directory + result_file
with open(output_file_path, 'w', newline='', encoding="utf-8") as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    for paper in os.listdir(source_directory):
        path = source_directory + paper
        if os.path.isdir(path):
            print("\n\n" + paper)
            tex_files = [x for x in os.listdir(path) if x.endswith('.tex')]
            print(tex_files)

            complete_tex = ""
            for file in tex_files:
                try:
                    f = open(path + "/" + file, "r", encoding="utf8")
                    complete_tex += f.read()
                    f.close()
                except UnicodeDecodeError as e:
                    print("UnicodeDecodeError occurred. File could not be loaded.")
                    continue
            #print("Complete tex source:\n" + complete_tex)

            '''
            Different LATEX tables:
                -table
                -table*
                -tabular
                -tabularx
            '''
            found_title = re.search(r"\\title\{(.*)\}", complete_tex)
            found_document_header = re.search(r"\\documentclass\[.*\\begin\{document\}", complete_tex, re.DOTALL)
            paper_id = str(uuid.uuid4())
            if found_document_header and found_title:
                #print(found_title.group(1))
                #print(found_document_header.group(0))
                
                paper_header_file = table_header_directory + paper_id + ".txt"
                f = open(paper_header_file, "w")
                f.write(found_document_header.group(0))
                f.close()
            found_tables = re.findall(r"\\begin\{table\*?\}.*?\\end\{table\*?\}", complete_tex, re.DOTALL)
            for table in found_tables:
                #r"\\caption\{(.*?)}"
                caption_match = re.search(r"\\caption\{(.*)}", table)
                if caption_match:
                    found_caption = caption_match.group(1)
                    original_caption = found_caption
                    full_label_match = re.search(r"\\label\{.*?\}", found_caption)
                    half_label_match = re.search(r"\\label\{.*", found_caption)
                    if full_label_match:
                        found_caption = found_caption.replace(full_label_match.group(0), "")
                        #print("Found full label: " + full_label_match.group(0))
                    elif half_label_match:
                        found_caption = found_caption.replace(half_label_match.group(0), "")
                        #print("Found half label: " + half_label_match.group(0))
                    print("Found caption: " + found_caption)
                    table_id = str(uuid.uuid4())
                    
                    spamwriter.writerow([table_id, paper_id, found_caption, original_caption])
                    table_file_path = table_code_directory + table_id + ".txt"
                    f = open(table_file_path, "w")
                    f.write(table)
                    f.close()
                    
                #print("Found table: \n" + table + "\n\n")

        break



2001.00116v2
['sample-sigconf.tex', 'table_1.tex', 'table_2.tex', 'table_3.tex', 'table_5.tex', 'table_6.tex', 'table_7.tex']
Found caption: Clusters splitting result.}
Found caption: Performance of \oursys. After \oursys is trained using training datasets that contain benign samples, CW and DeepFool AEs, \fedit{the detection rate and FPR (the rate of benign samples misclassified as AEs) are measured using testing sets.}}
Found caption: Comparison with other AE detectors (DR: Detection Rate). We use the same attack settings as used in prior work~\cite{ma2019nic,xu2017feature}.}
Found caption: Target-model agnostic property of \oursys.}
Found caption: Performance of integrating \oursys with an existing  detector~\cite{zuo2019l0}.}
Found caption: Impacts of different values of $n$ (CIFAR-10).}
Found caption: Impacts of different values of $n$ (ImageNet).}
Found caption: Impacts of different values of $n$.}
